# MNIST Image classification using CNN in TensorFlow

## 1. Import Modules

In [ ]:
# Import TensorFlow
import tensorflow as tf
# Import MNIST dataset
from tensorflow.examples.tutorials import mnist
# Import numpy
import numpy as np
# Import matplotlib
import matplotlib.pyplot as plt

## 2. Read in MNIST data

In [ ]:
# Read data
data = mnist.input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
# Shapes of training set
print("Training set (images) shape: {shape}".format(shape=data.train.images.shape))
print("Training set (labels) shape: {shape}".format(shape=data.train.labels.shape))

# Shapes of test set
print("Test set (images) shape: {shape}".format(shape=data.test.images.shape))
print("Test set (labels) shape: {shape}".format(shape=data.test.labels.shape))

image_hgt = 28
image_wid = 28
n_classes = 10

In [ ]:
# Create dictionary of target classes
label_dict = {
    0: 'Zero',
    1: 'One',
    2: 'Two',
    3: 'Three',
    4: 'Four',
    5: 'Five',
    6: 'Six',
    7: 'Seven',
    8: 'Eight',
    9: 'Nine'
}

In [ ]:
plt.figure(figsize=[5,5])

# Display first image in training data
plt.subplot(121)
img = np.reshape(data.train.images[0], (image_hgt,image_wid))
lbl = np.argmax(data.train.labels[0,:])
plt.imshow(img, cmap='gray')
plt.title("(Label: " + str(label_dict[lbl]) + ")")

# Display second image in testing data
plt.subplot(122)
img = np.reshape(data.test.images[1], (image_hgt,image_wid))
lbl = np.argmax(data.test.labels[1,:])
plt.imshow(img, cmap='gray')
plt.title("(Label: " + str(label_dict[lbl]) + ")")

## 3. Data preprocessing

In [ ]:
# Reshape training and testing images
train_X = data.train.images.reshape(-1, image_hgt, image_wid, 1) # number of samples, length, width, number of channels
test_X = data.test.images.reshape(-1, image_hgt, image_wid, 1) # number of samples, length, width, number of channels
print("Shape of training data: ", train_X.shape)
print("Shape of testing data: ", test_X.shape)
print("Type of training data element: ", type(train_X[0][0][0][0]))
print("Type of testing data element: ", type(test_X[0][0][0][0]))

train_Y = data.train.labels
test_Y = data.test.labels
print("Shape of training label: ", train_Y.shape)
print("Shape of testing lab: ", test_Y.shape)

## 4. Training Parameters

In [ ]:
learning_rate = 0.001
epochs = 50
batch_size = 64

## 5. Model

### 5.1. Helper Functions

In [ ]:
# Weights dictionary        
weight_dict = {
    'filterWC1' : tf.get_variable('FWC1', shape=(3,3,1,4), initializer=tf.contrib.layers.xavier_initializer()),
    'filterWC2' : tf.get_variable('FWC2', shape=(3,3,4,8), initializer=tf.contrib.layers.xavier_initializer()),
    'fullyConnW' : tf.get_variable('FCW', shape=(7*7*8,8), initializer=tf.contrib.layers.xavier_initializer()),
    'OutputW' : tf.get_variable('OW', shape=(8,n_classes), initializer=tf.contrib.layers.xavier_initializer())
}

bias_dict = {
    'filterBC1' : tf.get_variable('FBC1', shape=(4), initializer=tf.contrib.layers.xavier_initializer()),
    'filterBC2' : tf.get_variable('FBC2', shape=(8), initializer=tf.contrib.layers.xavier_initializer()),
    'fullyConnB' : tf.get_variable('FCB', shape=(8), initializer=tf.contrib.layers.xavier_initializer()),
    'OutputB' : tf.get_variable('OB', shape=(n_classes), initializer=tf.contrib.layers.xavier_initializer())
}

### 5.2. Network

In [ ]:
def conv_net(inp, weight_dict, bias_dict):
    # Convolution Layer 1
    conv1 = tf.nn.conv2d(inp, weight_dict['filterWC1'], strides = [1, 1, 1, 1], padding = 'SAME')
    conv1 = tf.nn.bias_add(conv1, bias_dict['filterBC1'])
    conv1 = tf.nn.leaky_relu(conv1, 0.1)
    
    conv1 = tf.nn.max_pool(conv1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    
    # Convolution Layer 2
    conv2 = tf.nn.conv2d(conv1, weight_dict['filterWC2'], strides = [1, 1, 1, 1], padding = 'SAME')
    conv2 = tf.nn.bias_add(conv1, bias_dict['filterBC2'])
    conv2 = tf.nn.leaky_relu(conv1, 0.1)
    
    conv2 = tf.nn.max_pool(conv2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    
    # Flatten the convolution layer output
    flatten = tf.reshape(conv2, [-1, weight_dict['fullyConnW'].get_shape().as_list()[0]])
    
    # Fully Connected Layer
    fc = tf.matmul(flatten, weight_dict['fullyConnW'])
    fc = tf.add(fc, bias_dict['fullyConnB'])
    fc = tf.nn.leaky_relu(fc, 0.1)
    
    # Output, class prediction
    out = tf.matmul(fc, weight_dict['OutputW'])
    out = tf.add(out, bias_dict['OutputB'])
    out = tf.nn.softmax(out);
    
    return out
    

### 5.3. Loss and Optimizer

In [ ]:
# Define placeholders
x = tf.placeholder(tf.float32, [None, image_hgt, image_wid, 1])
y = tf.placeholder(tf.float32, [None, n_classes])

In [ ]:
# Prediction
pred = conv_net(x, weight_dict, bias_dict)

# Define the Cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

# Define the optimizer
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### 5.4. Model Evaluation

In [ ]:
# Here you check whether the index of the maximum value of the predicted image 
# is equal to the actual labelled image and both will be a column vector
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

# Calculate accuracy across all the given images and average them out. 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## 6. TensorFlow session

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    # Variable must be initialized before a graph is used for the first time.
    sess.run(init)
    train_loss_list = []
    test_loss_list = []
    train_accuracy_list = []
    test_accuracy_list = []
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    
    for epoch in range(epochs):
        for batch in range(len(train_X)//batch_size):
            batch_x = train_X[batch*batch_size:min(((batch+1)*batch_size), len(train_X))]
            batch_y = train_Y[batch*batch_size:min(((batch+1)*batch_size), len(train_Y))]
            #print(batch_x.shape, batch_y.shape)
            opt = sess.run(optimizer, feed_dict = {x:batch_x, y:batch_y})
            train_loss, train_acc = sess.run([cost, accuracy], feed_dict = {x:batch_x, y:batch_y})
            #print("Epoch " + str(epoch) + ", Batch " + str(batch) + ", Loss = {:.6f}".format(train_loss) + ", Training Accuracy = {:.5f}".format(train_acc))
            
        print("Epoch " + str(epoch) + ", Loss = {:.6f}".format(train_loss) + ", Training Accuracy = {:.5f}".format(train_acc))
        
        
        # Calclate accuracy for all test images
        test_loss, test_acc = sess.run([cost, accuracy], feed_dict = {x:test_X, y:test_Y})
        print("Testing Accuracy: {:.5f}".format(test_acc))
        
        train_loss_list.append(train_loss)
        test_loss_list.append(test_loss)
        train_accuracy_list.append(train_acc)
        test_accuracy_list.append(test_acc)
        
    summary_writer.close()        

## 7. Visualize Loss

In [ ]:
plt.plot(range(epochs), train_loss_list, 'b', label='Training Loss')
plt.plot(range(epochs), test_loss_list, 'r', label='Test Loss')
plt.title('Training and Test Loss')
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.legend()
plt.figure()
plt.show()

plt.plot(range(epochs), train_accuracy_list, 'b', label='Training Accuracy')
plt.plot(range(epochs), test_accuracy_list, 'r', label='Test Accuracy')
plt.title('Training and Test Accuracy')
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.legend()
plt.figure()
plt.show()